In [2]:
import numpy as np
import os
import sys
import json


from dotenv import load_dotenv
load_dotenv()


True

## LLM Usage

In [3]:
import openai
client = openai.OpenAI()  # This uses your OPENAI_API_KEY from env

In [21]:
# question = "What are the top factors in my life that contribute to my objective and subjective health? Keep it medium burden, 4 weeks"
question = "How does my mood affect my sleep?"

system_prompt = open("experiment_prompt.txt", "r").read()
with open("library.json", "rb") as f:
    library_obj = json.load(f)
with open("schema.json", "rb") as f:
    schema_obj = json.load(f)


user_request = (
    system_prompt
        .replace("{{QUESTION}}", question)
        .replace("{{LIBRARY_JSON}}", json.dumps(library_obj, ensure_ascii=False))
        .replace("{{SCHEMA_JSON}}", json.dumps(schema_obj, ensure_ascii=False))
)

In [23]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": user_request}
    ],
    temperature=0
)
# Extract and parse the JSON from the LLM's response
content = response.choices[0].message.content
result = json.loads(content)

In [26]:
result

{'metadata': {'generated_at': '2025-08-18T12:00:00Z',
  'planner_version': '1.0.0',
  'input_question': 'How does my mood affect my sleep?',
  'library_version': '0.1.0'},
 'title': 'Mood and Sleep Self-Tracking Experiment',
 'research_question': 'How does mood impact sleep quality and duration?',
 'hypotheses': ['Higher positive mood leads to better sleep quality and longer duration.',
  'Increased stress levels result in poorer sleep quality and shorter duration.'],
 'sample': {'size': 1, 'population': 'Self'},
 'design': {'type': 'observational',
  'duration_days': 28,
  'participant_burden_minutes_per_day': 12},
 'instruments': [{'library_id': 'survey.daily_core.v1',
   'name': 'Daily Core Survey',
   'modality': 'survey',
   'schedule': {'frequency': 'daily', 'times_of_day': ['morning']},
   'metrics': ['energy', 'happiness', 'stress'],
   'rationale': 'To capture daily mood and stress levels which may impact sleep.'},
  {'library_id': 'survey.behavioral.v1',
   'name': 'Behaviora

In [31]:
sap_prompt = open("plan_to_sap_prompt.txt", "r").read()
sap_request = (
    sap_prompt
        .replace("{{QUESTION}}", question)
        .replace("{{EXP_JSON}}", json.dumps(result, ensure_ascii=False))
        .replace("{{SCHEMA_JSON}}", json.dumps(schema_obj, ensure_ascii=False))
)

sap_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": sap_request}
    ],
    temperature=0
)
# Extract and parse the JSON from the LLM's response
sap_contest = sap_response.choices[0].message.content
# result = json.loads(content)

In [35]:
print(sap_contest)

## Title
- Mood and Sleep Self-Tracking Experiment

## Research Question & Hypotheses
- Research Question: How does mood impact sleep quality and duration?
- Hypotheses:
  - Higher positive mood leads to better sleep quality and longer duration.
  - Increased stress levels result in poorer sleep quality and shorter duration.

## Sample Characteristics
- Sample Size: 1
- Population: Self
- Inclusion Criteria: N/A
- Exclusion Criteria: N/A

## Study Design
- Design Type: Observational
- Duration: 28 days
- Timing Windows: Morning, Evening
- Burden: 12 minutes per day, average

## Instruments & Rationale
- Daily Core Survey (survey.daily_core.v1 / survey)
  - Schedule: Daily, Morning
  - Metrics: energy, happiness, stress
  - Rationale: To capture daily mood and stress levels which may impact sleep.
- Behavioral Survey (survey.behavioral.v1 / survey)
  - Schedule: Daily, Evening
  - Metrics: sleep duration, sleep quality
  - Rationale: To track sleep duration and quality in relation to da

In [30]:
json.dumps(result, ensure_ascii=False)

'{"metadata": {"generated_at": "2025-08-18T12:00:00Z", "planner_version": "1.0.0", "input_question": "How does my mood affect my sleep?", "library_version": "0.1.0"}, "title": "Mood and Sleep Self-Tracking Experiment", "research_question": "How does mood impact sleep quality and duration?", "hypotheses": ["Higher positive mood leads to better sleep quality and longer duration.", "Increased stress levels result in poorer sleep quality and shorter duration."], "sample": {"size": 1, "population": "Self"}, "design": {"type": "observational", "duration_days": 28, "participant_burden_minutes_per_day": 12}, "instruments": [{"library_id": "survey.daily_core.v1", "name": "Daily Core Survey", "modality": "survey", "schedule": {"frequency": "daily", "times_of_day": ["morning"]}, "metrics": ["energy", "happiness", "stress"], "rationale": "To capture daily mood and stress levels which may impact sleep."}, {"library_id": "survey.behavioral.v1", "name": "Behavioral Survey", "modality": "survey", "sch